# Azure ML Python SDK ile AutoML Kullanımı
Gereklilikler:
* Python 3.6.1 ve üzeri
* Azure ML SDK (`!pip install azureml-sdk`) + (`!pip install azureml-sdk`). yükseltmek için: (`!pip install --upgrade azureml-sdk azureml-widgets`)
* Azure ML Workspace (Azure Portal üzerinden oluşturulabilir)
* config.json (Azure ML Workspace hakkında bilgi tutan ve workspace'in overview penceresinden indirilen dosya)
* BikeBuyer veri seti.(csv formatında, müşteri özellikleri ve alışveriş yapıp yapmadıkları bilgisi tutuluyor)

In [1]:
import azureml.core
print(azureml.core.VERSION)

1.19.0


## Workspace'i SDK ile oluşturmak ve yakalamak

In [2]:
from azureml.core import Workspace

In [3]:
# from_config ile config dosyasını kullanarak workspace yakalayalım
ws=Workspace.from_config()
#ws.get_details() detayları öğrenmek için

## Azure ML Datastore'a dosya upload etme Dataset oluşturma
* csv dosyasını Datastore (azureml tarafında azure diski) upload edelim
* Bu dosyadan Dataset oluşturalım

#### csv dosyasına DataFrame olarak inceleyelim

In [4]:
# csv içeriğine bakalım
import pandas as pd

In [5]:
df=pd.read_csv("TargetMail.csv")
print(df.shape)
#print(df.columns)
#df.head()
#df.isnull().any()
#df["BikeBuyer"].unique()

(18484, 14)


#### Dosyayı upload edelim

In [6]:
#Default Datastoreu yakalayalım
ds=ws.get_default_datastore()

In [7]:
#sadece yerel dosyaları buluta göndermek
dosyalar=["TargetMail.csv"]
ds.upload_files(
    files=dosyalar,
    target_path="customdatafile",
    overwrite=True,
    show_progress=True,
)

Uploading an estimated of 1 files
Uploading TargetMail.csv
Uploaded TargetMail.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_72632d032d25484bb863e11e9e465468

#### Dataset oluşturalım

In [8]:
from azureml.core import Dataset

In [ ]:
csv_files=[(ds,"customdatafile/*.csv")] #tüm dosyalar

#dataset nesnesi oluşturalım
ds_tabular=Dataset.Tabular.from_delimited_files(path=csv_files)

#dataseti register edelim
ds_tabular.register(workspace=ws,
                    name="TargetMailDSwSDK",)

#### Register edilmiş Dataseti yakalayalım

In [10]:
#Workspace koleksiyonundan alabiliriz.
ds_AdwTargetMail=ws.datasets["TargetMailDSwSDK"]

#direk adıyla çağırabiliriz.
#ds_AdwTargetMail=Dataset.get_by_name(ws,"TargetMailDSwSDK")

In [11]:
#Azure Dataseti pandas dataframe e çevirelim.
df=ds_AdwTargetMail.to_pandas_dataframe()
df.head()

,CustomerKey,Age,CommuteDistance,EnglishEducation,Gender,HouseOwnerFlag,MaritalStatus,NumberCarsOwned,NumberChildrenAtHome,EnglishOccupation,Region,TotalChildren,YearlyIncome,BikeBuyer
0,11000,51,1-2 Miles,Bachelors,M,1,M,0,0,Professional,Pacific,2,90000,1
1,11001,52,0-1 Miles,Bachelors,M,0,S,1,3,Professional,Pacific,3,60000,1
2,11002,52,2-5 Miles,Bachelors,M,1,M,1,3,Professional,Pacific,3,60000,1
3,11003,50,5-10 Miles,Bachelors,F,0,S,1,0,Professional,Pacific,0,70000,1
4,11004,49,1-2 Miles,Bachelors,F,1,S,4,5,Professional,Pacific,5,80000,1


## AutoMLConfig, Experiment ve Run oluşturma

#### Compute yakalayalım veya oluşturalım

In [12]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

In [13]:
cluster_name = "ktc"

try:
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Böyle bir compute mevcut! Kullanabilirsin')
except ComputeTargetException:
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', min_nodes=1, max_nodes=1)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Böyle bir compute mevcut! Kullanabilirsin


#### AutoMLConfig oluşturalım

In [14]:
from azureml.train.automl import AutoMLConfig

In [15]:
# Kullanılabilecek primary metricleri görmek için
from azureml.train.automl.utilities import get_primary_metrics

get_primary_metrics("classification")

['average_precision_score_weighted',
 'norm_macro_recall',
 'accuracy',
 'precision_score_weighted',
 'AUC_weighted']

In [17]:
#dir(ds_train) #attributelerine bakabiliriz.

#Dataseti test ve train olarak ayıralım.
ds_train, ds_test =ds_AdwTargetMail.random_split(percentage=0.7)

In [18]:
autml_config=AutoMLConfig(name="SDK ile AutoML",
                          task="classification",
                          primary_metric="accuracy",
                          compute_target=training_cluster,
                          training_data=ds_train,
                          validation_data=ds_test,
                          label_column_name="BikeBuyer",
                          featurization="auto",
                          iterations=5,
                          max_concurrent_iterations=2,
                          experiment_timeout_hours=0.25, #15 dk
                          experiment_exit_score=0.80
                         )

#### Experiment oluşturalım ve AutoMLConfig submit edelim

In [19]:
from azureml.core.experiment import Experiment

In [20]:
exp=Experiment(ws,'exp_automl_sdk')

In [21]:
run=exp.submit(autml_config)

Running on remote.


#### RunDetails widget ile çalışmayı takip edelim ve bitene kadar bekleyelim

In [22]:
from azureml.widgets import RunDetails

In [ ]:
RunDetails(run).show()
run.wait_for_completion(show_output=True)

#### En iyi çalışma ve en iyi modeli yakalayalım
modeli almak için `pip install azureml-train-automl-runtime==1.19.0`

In [24]:
best_run, fitted_model = run.get_output()
print(best_run,fitted_model,"\n")

Run(Experiment: exp_automl_sdk,
Id: AutoML_501422c0-fbfc-4620-a43c-5d404fe1f871_0,
Type: azureml.scriptrun,
Status: Completed) Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('MaxAbsScaler', MaxAbsScaler(copy...
                 LightGBMClassifier(boosting_type='gbdt', class_weight=None,
                                    colsample_bytree=1.0,
                                    importance_type='split', learning_rate=0.1,
                                    max_depth=-1,

In [25]:
best_run.get_metrics()

{'log_loss': 0.4044325455659358,
 'AUC_micro': 0.9136934245484021,
 'AUC_macro': 0.9136893211393466,
 'precision_score_macro': 0.8363915637218597,
 'norm_macro_recall': 0.6728911586127833,
 'accuracy': 0.8364067920394377,
 'precision_score_weighted': 0.8364636444250617,
 'f1_score_micro': 0.8364067920394377,
 'matthews_correlation': 0.6728371408600571,
 'average_precision_score_micro': 0.9107182455220276,
 'AUC_weighted': 0.9136893211393468,
 'precision_score_micro': 0.8364067920394377,
 'recall_score_macro': 0.8364455793063916,
 'average_precision_score_weighted': 0.9109923440258016,
 'balanced_accuracy': 0.8364455793063916,
 'recall_score_weighted': 0.8364067920394377,
 'average_precision_score_macro': 0.9109878389328883,
 'f1_score_macro': 0.8363967077925887,
 'recall_score_micro': 0.8364067920394377,
 'f1_score_weighted': 0.8364133585257578,
 'weighted_accuracy': 0.8363680178064747,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_501422c0-fbfc-4620-a43c-5d404fe1f871_

In [26]:
#en iyi modelin adımları görebiliriz.
for step in fitted_model.named_steps:
    print(step)

datatransformer
MaxAbsScaler
LightGBMClassifier


## Modeli register etmek ve indirmek

#### Modeli Register etmek

In [27]:
#run üzerinden de model register edilebilir.
best_run.register_model(model_name="ModelBikeBuyerwSDK",
                        model_path="outputs/model.pkl") #AutoML sayfasından modelin outputs+logs tabında

Model(workspace=Workspace.create(name='kisemlws', subscription_id='b936214e-de5a-41d9-92ac-f79505ab5fd4', resource_group='krg'), name=ModelBikeBuyerwSDK, id=ModelBikeBuyerwSDK:1, version=1, tags={}, properties={})

In [28]:
from azureml.core.model import Model

#### Modeli listelemek ve indirmek

In [29]:
# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)

ModelBikeBuyerwSDK version: 1
AutoMLa8c8ff3ab0 version: 1


In [31]:
#modeli indirmek için
model=Model(ws,"ModelBikeBuyerwSDK")
model.download(target_dir=".",exist_ok=True)
#model.delete()  #register edilen modeli silmek için

#kontrol edelim var mı
import os
os.stat("./model.pkl")

os.stat_result(st_mode=33279, st_ino=9655805562012565504, st_dev=46, st_nlink=1, st_uid=0, st_gid=0, st_size=374025, st_atime=1610530261, st_mtime=1610530261, st_ctime=1610530261)

#### Modelden bir webservice oluşturmak ve test kısmı ile devam eder.

* score.py(init ve run) ve yml dosyasından
* Container config tanımla
* Yukırıdakilerden image oluştur ve containera deploy et.
* Webservice(ws,"serviceadi") veya request sınıf ile test et.
    